In [1]:
from __future__ import print_function
%matplotlib inline
import os
import warnings 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as image
import pandas as pd
import pandas_profiling
plt.style.use("ggplot")
warnings.simplefilter("ignore")
plt.rcParams['figure.figsize'] = (12,8)

In [2]:
hr = pd.read_csv("data/employee_data.csv")
hr.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,quit,promotion_last_5years,department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [ ]:
x=hr.profile_report(title ="Data Report")
x.to_file("output.html")
x

In [ ]:
pd.crosstab(hr.salary , hr.quit).plot(kind = 'bar')
plt.title("Turnover Frequency on Salary Bracket")
plt.xlabel("Salary")
plt.ylabel("Frequency of turnover")
plt.show()

In [ ]:
pd.crosstab(hr.department , hr.quit).plot(kind = 'bar')
plt.title("Turnover Frequency on department Bracket")
plt.xlabel("Department")
plt.ylabel("Frequency of turnover")
plt.show()

In [ ]:
pd.crosstab(hr.time_spend_company , hr.quit).plot(kind = 'bar')
plt.title("Turnover Frequency on time_spend_company Bracket")
plt.xlabel("time_spend_company")
plt.ylabel("Frequency of turnover")
plt.show()

In [ ]:
cat_vars  = ['department', 'salary']
for var in cat_vars:
    cat_list = pd.get_dummies(hr[var], prefix=var)
    hr = hr.join(cat_list)

In [ ]:
hr.head()

In [ ]:
hr.shape

In [ ]:
hr.drop(['department', 'salary'],1, inplace=True)

In [ ]:
from yellowbrick.target import ClassBalance
plt.style.use('ggplot')
plt.rcParams['figure.figsize']=(12,8)

In [ ]:
visualizer = ClassBalance(labels = ['stayed','quit']).fit(hr.quit)
visualizer.show()

In [ ]:
x = hr.loc[:,hr.columns!="quit"]
y = hr.quit

In [ ]:
x.head(5)

In [ ]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test  = train_test_split(x,y,test_size = 0.2 , random_state = 0 , stratify = y)

In [ ]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import export_graphviz # display the tree within a Jupyter notebook
from IPython.display import SVG
from graphviz import Source
from IPython.display import display
from ipywidgets import interactive, IntSlider, FloatSlider, interact
import ipywidgets
from IPython.display import Image
from subprocess import call
import matplotlib.image as mpimg

In [ ]:
@interact
def plot_tree_fr(
    crit = ['gini','entropy'],
    bootstrap = ['True','False'],
    depth = IntSlider(min=1,max=30,value=3,continuous_update=False),
    min_split =  IntSlider(min=2,max=5,value=2,continuous_update=False),
    min_leaf =  IntSlider(min=1,max=5,value=1,continuous_update=False),
    forests =  IntSlider(min=1,max=200,value=100,continuous_update=False) 
):
    estimator = RandomForestClassifier(random_state=1,
                                      criterion=crit,
#                                       splitter=split,
                                       n_estimators=forests,
                                       max_depth=depth,
                                       min_samples_split= min_split,
                                       min_samples_leaf= min_leaf ,
                                       n_jobs=-1,
                                       verbose = False
                                      )
    estimator.fit(x_train , y_train)
    print('Random Forests  Training Accuracy : {:.3f}'.format(accuracy_score(y_train , estimator.predict(x_train))))
    print('Random Forests  Test Accuracy : {:.3f}'.format(accuracy_score(y_test , estimator.predict(x_test))))
    num_tree = estimator.estimators_[0]
    print("\Visualising Tree: ",0)
    
    graph = Source(tree.export_graphviz(num_tree,out_file=None , feature_names = x_train.columns,
                                       class_names =['stayed','quit'],filled=True))
    display(Image(data = graph.pipe(format = 'png')))
    return estimator

In [ ]:
from yellowbrick.model_selection import FeatureImportances
plt.rcParams['figure.figsize'] = (12,8)
plt.style.use("ggplot")

In [ ]:
rf = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=3, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=1,
                       verbose=False, warm_start=False)
viz = FeatureImportances(rf)
viz.fit(x_train , y_train)

In [ ]:
from yellowbrick.classifier import ROCAUC

vis = ROCAUC(rf , classes = ['stayed','quit'])

vis.fit(x_train , y_train)
vis.score(x_test , y_test)
vis.poof()